# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [20]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
import geoviews.tile_sources as gts
import hvplot.pandas

# Import API key
from api_keys import geoapify_key

In [21]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,iqaluit,63.7506,-68.5145,0.85,74,75,5.66,CA,2024-06-10 02:46:59
1,1,arizona city,32.7559,-111.6710,35.12,11,67,0.89,US,2024-06-10 02:46:59
2,2,naldurg,17.8167,76.3000,27.28,64,34,6.76,IN,2024-06-10 02:47:00
3,3,aweil,8.7682,27.4002,23.20,85,100,1.44,SS,2024-06-10 02:47:00
4,4,galliano,29.4422,-90.2992,27.78,77,2,2.91,US,2024-06-10 02:47:00


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [22]:
%%capture --no-display

# Configure the map plot
humidity_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.8
)

# Display the map
humidity_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [23]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df['Max Temp'] <= 29)
                                & (city_data_df['Max Temp'] >= 15)
                                & (city_data_df['Cloudiness'] <= 65)
                                & (city_data_df['Humidity'] <= 62)
                                & (city_data_df['Country'] != 'IN')
                                & (city_data_df['Country'] != 'IR')
                                & (city_data_df['Country'] != 'RU')
                                ]

# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,23,urumqi,43.8010,87.6005,27.53,26,0,2.00,CN,2024-06-10 02:43:19
34,34,yellowknife,62.4560,-114.3525,16.29,41,40,2.57,CA,2024-06-10 02:47:07
35,35,qamdo,31.1667,97.2333,16.21,38,21,1.46,CN,2024-06-10 02:47:07
53,53,port alfred,-33.5906,26.8910,15.37,59,13,3.02,ZA,2024-06-10 02:47:11
57,57,pangai,-19.8000,-174.3500,26.06,57,40,7.20,TO,2024-06-10 02:47:11


### Step 3: Create a new DataFrame called `hotel_df`.

In [24]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.iloc[:, [1, 8, 2, 3, 5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,iqaluit,CA,63.7506,-68.5145,74,
1,arizona city,US,32.7559,-111.6710,11,
2,naldurg,IN,17.8167,76.3000,64,
3,aweil,SS,8.7682,27.4002,85,
4,galliano,US,29.4422,-90.2992,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [25]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    'limit':1,
    'filter':'',
    'bias':'',
    'apiKey':geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
iqaluit - nearest hotel: Frobisher Inn
arizona city - nearest hotel: No hotel found
naldurg - nearest hotel: Residence Inn
aweil - nearest hotel: Aweil Grand Hotel
galliano - nearest hotel: Fairfield Inn & Suites Cut Off-Galliano
hawaiian paradise park - nearest hotel: No hotel found
saipan - nearest hotel: Chalan Kanoa Beach Hotel
mejit - nearest hotel: No hotel found
san jose de jachal - nearest hotel: San Martin
yazd - nearest hotel: هتل سنتی مهر
hithadhoo - nearest hotel: Pebbles Inn
labytnangi - nearest hotel: Семь лиственниц
hermanus - nearest hotel: Aloe guest house
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
nyurba - nearest hotel: Марина
saint-francois - nearest hotel: Chez Lily
rio grande - nearest hotel: Hotel Vila Moura Executivo
dudinka - nearest hotel: Виктория
longyearbyen - nearest hotel: The Vault
adamstown - nearest hotel: No hotel found
puerto natales - nearest hotel: Dorotea Patagonia Hostel
ceska kamenice - nearest hotel: Slav

,City,Country,Lat,Lng,Humidity,Hotel Name
0,iqaluit,CA,63.7506,-68.5145,74,Frobisher Inn
1,arizona city,US,32.7559,-111.6710,11,No hotel found
2,naldurg,IN,17.8167,76.3000,64,Residence Inn
3,aweil,SS,8.7682,27.4002,85,Aweil Grand Hotel
4,galliano,US,29.4422,-90.2992,77,Fairfield Inn & Suites Cut Off-Galliano
...,...,...,...,...,...,...
566,kralendijk,BQ,12.1500,-68.2667,79,Hotel Bonaire Inn
567,kristiansund,NO,63.1115,7.7320,87,Comfort Hotel Fosna
568,mont-dore,NC,-22.2833,166.5833,99,Les Cases de Plum
569,ouesso,CG,1.6136,16.0517,95,Auberge Nianina


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= 1.2)

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)